# Introduction to DataFrames
**[Bogumił Kamiński](http://bogumilkaminski.pl/about/), Dec 2, 2017**

A brief introduction to basic usage of `DataFrames`. Tested under version `0.11`.
I will try to keep it up to date as the package evolves.

In [2]:
using DataFrames # load package

## Joining DataFrames

### Cross join

In [4]:
# cross-join does tor require on argument
# it produces a Cartesian product or arguments
function expand_grid(;xs...) # a simple replacement for expand.grid in R
    reduce((x,y) -> join(x, DataFrame(Pair(y...)), kind=:cross),
           DataFrame(Pair(xs[1]...)), xs[2:end])
end

expand_grid(a=[1,2], b=["a","b","c"], c=[true,false])

,a,b,c
1,1,a,true
2,1,a,false
3,1,b,true
4,1,b,false
5,1,c,true
6,1,c,false
7,2,a,true
8,2,a,false
9,2,b,true
10,2,b,false
